## Requirements


In [1]:
# Unable warnings
import os
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


#### Directory adjustment


In [2]:
from pathlib import Path
import sys

# Back to main folder
path = os.path.dirname(os.getcwd()) + "/"
os.chdir(path)
sys.path.append(path)


#### Data Processing


In [3]:
# ETL
import numpy as np
import pandas as pd
import multiprocessing


#### Natural language processing


In [4]:
from gensim.models import KeyedVectors
from gensim import models
import gensim


#### Models

In [5]:
# ML preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split


# Deep learnig model
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional
import tensorflow as tf


In [6]:
# Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# Train  metrics
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]


#### Tracking

In [7]:
import mlflow
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('Hate Speech')

<Experiment: artifact_location='./artifacts/1', creation_time=1665929754799, experiment_id='1', last_update_time=1665929754799, lifecycle_stage='active', name='Hate Speech', tags={}>

## Split dataset


In [8]:
# Get data
df = pd.read_csv("data/corpus/augmented_corpus_fortuna.csv")

# Set target and features
target = "label"
features = "text_nonstop"
count = f"length_{features}"
pos = len(df.query('label==1'))
neg = len(df.query('label==0'))


# Break apart dataset
X = df[features].values.astype("U")
y = df[target]

# Split train abd test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Set k-fold criteria
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

# Classes balancing
longest_text = df[count].max()
initial_bias = np.log([pos/neg])

weight_for_0 = (1 / neg) * (len(df) / 2.0)
weight_for_1 = (1 / pos) * (len(df) / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}


## Neural network

In [9]:
# Preset parameters
experiment_parameters = {"classifier": "LSTM",
                         "class_weight": class_weight,
                         "epochs": 200,
                         "units": 50,
                         "dropout": 0.4,
                         "recurrent_dropout": 0.2,
                         "kernel_initializer": 'glorot_uniform',
                         "loss": "binary_crossentropy",
                         "optimizer": "adamax",
                         "batch_size": 64}


#### Basic pipeline

In [10]:
from keras.layers import TextVectorization
from keras.layers import Input


In [11]:
tfidf_vectorizer = TextVectorization(max_tokens=1500, standardize="lower_and_strip_punctuation",
                                     split="whitespace", output_mode="tf-idf", ngrams=(1, 2))
tfidf_vectorizer.adapt(X_train, batch_size=32)
vocab = tfidf_vectorizer.get_vocabulary()
out = tfidf_vectorizer(X_train[:5])


In [12]:
# Simple Model
def nn_builder(text_vectorizer):
    nn = Sequential()
    nn.add(Input(shape=(1,), dtype="string"))
    nn.add(text_vectorizer)
    nn.add(Dense(500, activation="relu"))
    nn.add(Dropout(0.4))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(loss=experiment_parameters['loss'],
               optimizer=experiment_parameters['optimizer'],
               metrics=METRICS)
    return nn


nn_model = nn_builder(tfidf_vectorizer)

history = nn_model.fit(X_train, y_train,
                       experiment_parameters['batch_size'],
                       experiment_parameters['epochs'],
                       validation_data=(X_test, y_test),
                       callbacks=[EarlyStopping(monitor="loss",
                                                patience=10,
                                                restore_best_weights=True)])


Epoch 1/200
71/71 [==============================] - 2s 12ms/step - loss: 0.8993 - tp: 456.0000 - fp: 818.0000 - tn: 2289.0000 - fn: 973.0000 - accuracy: 0.6052 - precision: 0.3579 - recall: 0.3191 - auc: 0.5247 - prc: 0.3378 - val_loss: 0.5866 - val_tp: 79.0000 - val_fp: 25.0000 - val_tn: 752.0000 - val_fn: 278.0000 - val_accuracy: 0.7328 - val_precision: 0.7596 - val_recall: 0.2213 - val_auc: 0.6626 - val_prc: 0.5294
Epoch 2/200
71/71 [==============================] - 0s 5ms/step - loss: 0.7109 - tp: 614.0000 - fp: 678.0000 - tn: 2429.0000 - fn: 815.0000 - accuracy: 0.6709 - precision: 0.4752 - recall: 0.4297 - auc: 0.6319 - prc: 0.4611 - val_loss: 0.5626 - val_tp: 107.0000 - val_fp: 45.0000 - val_tn: 732.0000 - val_fn: 250.0000 - val_accuracy: 0.7399 - val_precision: 0.7039 - val_recall: 0.2997 - val_auc: 0.7049 - val_prc: 0.5744
Epoch 3/200
71/71 [==============================] - 0s 6ms/step - loss: 0.6543 - tp: 626.0000 - fp: 534.0000 - tn: 2573.0000 - fn: 803.0000 - accuracy: 0

In [13]:
test_preds = nn_model.predict(X_test)
pd.DataFrame(classification_report(
    y_test, np.where(test_preds >= 0.5, 1, 0), output_dict=True))


36/36 [==============================] - 0s 2ms/step


,0,1,accuracy,macro avg,weighted avg
precision,0.767189,0.537705,0.705467,0.652447,0.694944
recall,0.818533,0.459384,0.705467,0.638958,0.705467
f1-score,0.792030,0.495468,0.705467,0.643749,0.698668
support,777.000000,357.000000,0.705467,1134.000000,1134.000000


## Deep learning

#### Basic pipeline

In [14]:
# Deep learnig Model
def nn_builder(text_vectorizer):
    nn = Sequential()
    nn.add(Input(shape=(1,), dtype="string"))
    nn.add(text_vectorizer)
    nn.add(Dense(1500, activation="relu"))
    nn.add(Dense(500, activation="relu"))
    nn.add(Dropout(0.40))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(loss=experiment_parameters['loss'],
               optimizer=experiment_parameters['optimizer'],
               metrics=METRICS)
    return nn


deep_model = nn_builder(tfidf_vectorizer)

history = deep_model.fit(X_train, y_train,
                         experiment_parameters['batch_size'],
                         experiment_parameters['epochs'],
                         validation_data=(X_test, y_test))


Epoch 1/200
71/71 [==============================] - 2s 18ms/step - loss: 0.8792 - tp: 563.0000 - fp: 735.0000 - tn: 3149.0000 - fn: 1223.0000 - accuracy: 0.6547 - precision: 0.4337 - recall: 0.3152 - auc: 0.5824 - prc: 0.4022 - val_loss: 0.5867 - val_tp: 50.0000 - val_fp: 11.0000 - val_tn: 766.0000 - val_fn: 307.0000 - val_accuracy: 0.7196 - val_precision: 0.8197 - val_recall: 0.1401 - val_auc: 0.6608 - val_prc: 0.5103
Epoch 2/200
71/71 [==============================] - 1s 12ms/step - loss: 0.5494 - tp: 487.0000 - fp: 227.0000 - tn: 2880.0000 - fn: 942.0000 - accuracy: 0.7423 - precision: 0.6821 - recall: 0.3408 - auc: 0.7321 - prc: 0.5801 - val_loss: 0.5560 - val_tp: 120.0000 - val_fp: 47.0000 - val_tn: 730.0000 - val_fn: 237.0000 - val_accuracy: 0.7496 - val_precision: 0.7186 - val_recall: 0.3361 - val_auc: 0.7208 - val_prc: 0.5676
Epoch 3/200
71/71 [==============================] - 1s 13ms/step - loss: 0.4964 - tp: 672.0000 - fp: 258.0000 - tn: 2849.0000 - fn: 757.0000 - accuracy

In [15]:
test_preds = deep_model.predict(X_test)
pd.DataFrame(classification_report(
    y_test, np.where(test_preds >= 0.5, 1, 0), output_dict=True))


36/36 [==============================] - 0s 4ms/step


,0,1,accuracy,macro avg,weighted avg
precision,0.768409,0.554795,0.713404,0.661602,0.701160
recall,0.832690,0.453782,0.713404,0.643236,0.713404
f1-score,0.799259,0.499230,0.713404,0.649244,0.704805
support,777.000000,357.000000,0.713404,1134.000000,1134.000000
